## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBBSBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
823
+------------------+------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|          coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
|2806.9396821891055|1572.4537935573317|656.3199871987487|    299.0|BBBSBBBB| 42.4833|    -72.0|       SNWD|0.30381929180996703|0.08532712252644134|0.047206349786434974|0.676324121798292

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [14]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year      coeff_3
0  USC00193772  1969.0   656.319987
1  USC00196783  1969.0   442.182336
2  USC00193702  1961.0 -1093.605795
3  USC00196322  1987.0  -372.040548
4  USC00192806  1971.0   354.606681

In [15]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  USC00190120  USC00190166  USC00190408  USC00190535  USC00190562  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN          NaN   
1894.0           NaN          NaN          NaN          NaN          NaN   
1896.0           NaN          NaN          NaN          NaN          NaN   
1897.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   

station  USC00190593  USC00190998  USC00191323  USC00191344  USC00191622  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN          NaN   
1894.0           NaN          NaN          NaN          NaN          NaN   
1896.0           NaN          NaN          NaN          NaN          NaN   
1897.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00199928  USW00004769  USW00004779  USW00014702  \
year        ...                                                            
1893.0      ...               NaN          NaN          NaN          NaN   
1894.0      ...               NaN          NaN          NaN          NaN   
1896.0      ...               NaN          NaN          NaN          NaN   
1897.0      ...               NaN          NaN          NaN          NaN   
1898.0      ...               NaN          NaN          NaN          NaN   
1899.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   

station  USW00014703  USW00014739  USW00014753  USW00014790  USW00094701  \
year                                                                       
1893.0           NaN          NaN          NaN          NaN   483.013400   
1894.0           NaN          NaN          NaN          NaN  -351.247155   
1896.0           NaN          NaN          NaN          NaN   276.810755   
1897.0           NaN          NaN          NaN          NaN  -492.440011   
1898.0           NaN          NaN          NaN          NaN  -536.650285   
1899.0           NaN          NaN          NaN          NaN  -481.823793   
1902.0           NaN          NaN          NaN          NaN   328.377520   
1904.0           NaN          NaN          NaN          NaN   183.589754   
1905.0           NaN          NaN          NaN          NaN  -490.957822   
1907.0           NaN          NaN          NaN          NaN  -547.544685   

station  USW00094746  
year                  
1893.0           NaN  
1894.0           NaN  
1896.0           NaN  
1897.0           NaN  
1898.0           NaN  
1899.0           NaN  
1902.0           NaN  
1904.0           NaN  
1905.0           NaN  
1907.0           NaN  

[10 rows x 57 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [16]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  481.941668077
RMS removing mean-by-station=  472.904300187
RMS removing mean-by-year   =  161.470987643


In [13]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 907.115528379
0 after removing mean by year    = 533.110666707
0 after removing mean by stations= 426.931349026
1 after removing mean by year    = 410.60682172
1 after removing mean by stations= 406.596688586
2 after removing mean by year    = 405.247623375
2 after removing mean by stations= 404.688465264
3 after removing mean by year    = 404.426269021
3 after removing mean by stations= 404.293243109
4 after removing mean by year    = 404.221406416
4 after removing mean by stations= 404.18020512
